In [1]:
import pandas as pd

In [2]:
directory = '/Users/kwheatley/Desktop/Capstone/gcloud_data/'

In [3]:
# Load the list of relevant resume ids from 03_create_ngram_model
relevant_resume_ids = pd.read_csv(directory+'03_relevant_resume_ids.csv')
relevant_resume_ids.columns = ['resume_id']

In [4]:
current_job_titles = pd.read_csv(directory+'02_resumes_work.csv')
current_job_titles = current_job_titles\
    [current_job_titles.resume_id.isin(relevant_resume_ids.resume_id)]\
    [['resume_id','cleaned_job_title','from_year','to_year']]\
    .sort_values(by='from_year')
    
current_job_titles = current_job_titles.sort_values(by=['resume_id','from_year','to_year'])\
    .rename(columns={'cleaned_job_title': 'curr_cleaned_job_title'})
current_job_titles['next_cleaned_job_title'] = current_job_titles.groupby('resume_id')\
['curr_cleaned_job_title'].shift(-1).fillna("None")

nodes = pd.DataFrame(current_job_titles.curr_cleaned_job_title.unique()).reset_index()
nodes.columns = ['index_num','name']

current_job_titles = current_job_titles\
    [current_job_titles.curr_cleaned_job_title != current_job_titles.next_cleaned_job_title]
current_job_titles = current_job_titles\
    [current_job_titles.next_cleaned_job_title != 'None']
    
current_job_titles = current_job_titles\
    [['resume_id','curr_cleaned_job_title','next_cleaned_job_title']]

In [5]:
grouped_jobs = current_job_titles\
    .groupby(['curr_cleaned_job_title','next_cleaned_job_title'])\
    .resume_id.count().reset_index()

grouped_jobs = pd.merge(left=grouped_jobs,right=nodes, left_on='curr_cleaned_job_title', right_on='name')
del grouped_jobs['name']
grouped_jobs.columns = ['curr_cleaned_job_title','next_cleaned_job_title','value','source']
grouped_jobs = pd.merge(left=grouped_jobs,right=nodes, left_on='next_cleaned_job_title', right_on='name')
del grouped_jobs['name']
grouped_jobs.columns = ['curr_cleaned_job_title','next_cleaned_job_title','value','source','target']

In [6]:
del nodes['index_num']
links = grouped_jobs[['source','target','value']]

In [7]:
nodes_json = nodes.to_json(orient='records')
links_json = links.to_json(orient='records')

final = { '"nodes"' : nodes_json, '"links"' : links_json }
with open(directory+'06_job_sankey_data.json', 'w') as outfile:
    outfile.write(str(final).replace("\\","").replace("'",""))

# End